In [108]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
import time
import pandas as pd
import requests


# 2023 시즌 기록 크롤링

In [260]:
def get_page_df(table_soup,columns):
    row={}
    templen = len(table_soup.find_all("tr"))
    for i in range(2, templen):
        tempTr = table_soup.find_all("tr")[i]
        if(tempTr.find("th") is not None):
            continue
    
        column_idx = 0
        for j in range(len(columns)):
            #print(j)
            tempTd = tempTr.find_all("td")[j].text.strip()
            if(tempTd == "" and j > 0):
                continue
            row[columns[column_idx]] = row.get(columns[column_idx],[''])+[tempTd]
            column_idx += 1
    page_df = pd.DataFrame(row).iloc[1:,:]
    return page_df

In [261]:
page_df_dict = {}
tmp_list = []
columns = ['순', '이름', '팀','WAR', '게임', '타석', '타수', '득점', '안타', '2타', '3타', '홈런', '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비','타율','출루','장타','OPS','wOBA','wRC+']

for i in range(5):
    url = f'http://www.statiz.co.kr/stat.php?mid=stat&re=0&ys=2023&ye=2023&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR_ALL_ADJ&o2=TPA&de=1&lr=0&tr=&cv=&ml=2&sn=30&pa={i*30}&si=&cn='
    
    # 웹 페이지에 접속하여 HTML 내용 가져오기
    response = requests.get(url)
    html_content = response.text
    
    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(html_content, 'html.parser')
    table_soup = soup.find_all("table")[0]
    page_df = get_page_df(table_soup,columns)
    page_df_dict[i] = page_df


In [262]:
page_df_dict[4].head()

,순,이름,팀,WAR,게임,타석,타수,득점,안타,2타,...,삼진,병살,희타,희비,타율,출루,장타,OPS,wOBA,wRC+
1,121,이학주,23롯SS,0.21,104,122,110,22,23,2,...,31,2,0,0,.209,.287,.309,.596,.286,70.3
2,122,김동엽,23삼DH,0.21,69,182,165,20,42,7,...,29,8,0,2,.255,.313,.388,.701,.327,94.0
3,123,홍성호,23두RF,0.20,21,51,48,6,14,4,...,14,0,0,0,.292,.333,.375,.708,.335,103.0
4,124,문상철,23KDH,0.19,112,330,304,30,79,20,...,81,4,1,6,.260,.298,.414,.712,.324,92.4
5,125,신본기,23KSS,0.19,40,75,54,8,11,2,...,8,3,1,4,.204,.365,.296,.661,.322,92.1


# 2023 연봉 크롤링

In [323]:
def get_salary(url):

    sal_list = []
    # 첫 번째 url
    response = requests.get(url)
    html_content = response.text
    
    soup = BeautifulSoup(html_content, 'html.parser')
    all_a_tags  = soup.find_all("table")[2].find_all('a')
    for a_tag in all_a_tags:
        # 두 번째 url
        url1 = 'http://www.statiz.co.kr/' + a_tag.get('href')
    
        response = requests.get(url1)
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        tmp = soup.find_all('a', {'class': ['btn btn-app swipeclass']})
    
        # 세 번째 url
        url2 = 'http://www.statiz.co.kr/' + tmp[8].get('href')
    
        response = requests.get(url2)
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        tmp = soup.find_all('tr', {'class': ['oddrow_stz','evenrow_stz']})
        for i in range(len(tmp)):
            if tmp[i].find_all('td')[0].text == '2023':
                sal = tmp[i].find_all('td')[1].text
                #print(sal)
                #sal = int(sal.split(',')[0])*10e2
        sal_list.append(sal)
    return sal_list

In [324]:
for i in range(5):
    url = f'http://www.statiz.co.kr/stat.php?mid=stat&re=0&ys=2023&ye=2023&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR_ALL_ADJ&o2=TPA&de=1&lr=0&tr=&cv=&ml=2&sn=30&pa={i*30}&si=&cn='
    sal_list = get_salary(url)
    page_df_dict[i]['연봉'] = sal_list

In [330]:
page_df_dict[0]

,순,이름,팀,WAR,게임,타석,타수,득점,안타,2타,...,병살,희타,희비,타율,출루,장타,OPS,wOBA,wRC+,연봉
1,1,노시환,23한3B,6.93,131,595,514,85,153,30,...,13,0,3,.298,.388,.541,.929,.418,159.3,"12,000"
2,2,김혜성,23키2B,6.19,137,621,556,104,186,29,...,6,0,5,.335,.396,.446,.842,.392,142.1,"2,900"
3,3,최정,23S3B,5.78,128,552,471,94,140,31,...,12,1,6,.297,.388,.548,.936,.418,155.8,"120,000"
4,4,양의지,23두C,5.40,129,509,438,56,134,23,...,20,0,3,.306,.397,.475,.872,.399,151.9,"100,000"
5,5,오스틴,23L1B,5.22,139,583,520,87,163,29,...,11,0,7,.313,.376,.517,.893,.405,154.4,"3,000"
6,6,홍창기,23LRF,5.17,141,643,524,109,174,35,...,6,3,6,.332,.444,.412,.856,.407,152.0,"32,000"
7,7,구자욱,23삼RF,4.78,119,515,453,65,152,37,...,5,2,3,.336,.407,.494,.902,.414,153.2,"250,000"
8,8,소크라테스,23KCF,4.58,142,608,547,91,156,31,...,13,0,8,.285,.344,.463,.806,.366,125.7,"50,000"
9,9,손아섭,23NDH,4.39,140,609,551,97,187,36,...,10,1,5,.339,.393,.443,.836,.390,140.0,"150,000"
10,10,박건우,23NRF,4.25,130,533,458,70,146,34,...,14,1,9,.319,.397,.480,.877,.403,148.0,"190,000"


In [313]:
sal

12000.0

In [339]:
i=0
url = f'http://www.statiz.co.kr/stat.php?mid=stat&re=0&ys=2023&ye=2023&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR_ALL_ADJ&o2=TPA&de=1&lr=0&tr=&cv=&ml=2&sn=30&pa={i*30}&si=&cn='

# 웹 페이지에 접속하여 HTML 내용 가져오기
response = requests.get(url)
html_content = response.text

# BeautifulSoup을 사용하여 HTML 파싱
soup = BeautifulSoup(html_content, 'html.parser')
all_a_tags  = soup.find_all("table")[2].find_all('a')
# for a_tag in all_a_tags:
#     href_value = a_tag.get('href')
#     if href_value:
#         print(f'href: {href_value}')
url1 = 'http://www.statiz.co.kr/' + all_a_tags[0].get('href')

response = requests.get(url1)
html_content = response.text
soup = BeautifulSoup(html_content, 'html.parser')
tmp = soup.find_all('a', {'class': ['btn btn-app swipeclass']})
for i in tmp:
    #print(i.text)
    if i.text.strip()=='연봉':
        #print(i)
        print(i.get('href'))
# url2 = 'http://www.statiz.co.kr/' + tmp[8].get('href')

# response = requests.get(url2)
# html_content = response.text
# soup = BeautifulSoup(html_content, 'html.parser')
# tmp = soup.find_all('tr', {'class': ['oddrow_stz','evenrow_stz']})
# tmp
# for i in range(len(tmp)):
#     if tmp[i].find_all('td')[0].text == '2023':
#         sal = tmp[i].find_all('td')[1].text
#         sal = int(sal.split(',')[0])*10e2
#         print(sal)

player.php?opt=11&name=%EB%85%B8%EC%8B%9C%ED%99%98&birth=2000-12-03


In [295]:
10e2

1000.0

In [297]:
120000000 == 120000000.0

True

In [285]:
print(url2)


http://www.statiz.co.kr/player.php?opt=11&name=%EB%85%B8%EC%8B%9C%ED%99%98&birth=2000-12-03


In [263]:

# Chrome 웹 드라이버 경로
chrome_driver_path = 'chromedriver.exe'  # 본인의 ChromeDriver 경로로 수정

options = webdriver.ChromeOptions()
# # 창 숨기는 옵션 추가
# options.add_argument("headless")
# ChromeDriver 서비스 초기화
chrome_service = ChromeService(executable_path=chrome_driver_path)

# WebDriver 초기화
driver = webdriver.Chrome(service=chrome_service,options=options)

# Instagram 인기 게시물 페이지 열기
driver.get('http://www.statiz.co.kr/stat.php?mid=stat&re=0&ys=2023&ye=2023&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR_ALL_ADJ&o2=TPA&de=1&lr=0&tr=&cv=&ml=2&sn=30&pa=0&si=&cn=')

# 로딩을 위해 잠시 대기
time.sleep(2)

html = driver.page_source
soup  = BeautifulSoup(html, 'html.parser')
table_soup = soup.find_all("table")[1]

# # 인기 게시물 태그 찾기
# table = driver.find_elements(By.XPATH, "//div[@class='Nnq7C weEfm']")



# 브라우저 닫기
driver.quit()


In [188]:
tmp_list[0].find_all("tr")[2].find_all("td")[4].text.strip()

'131'

In [148]:
table_soup.find_all("tr")[2].find_all("td")[1].text.strip()

'노시환'

In [147]:
temp.find_all("tr")[2].find_all("td")[1].text.strip()

'노시환'

In [256]:
columns = ['순', '이름', '팀', '게임', '타석', '타수', '득점', '안타', '2타', '3타', '홈런', '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비','타율','출루','장타','OPS','wOBA']
df = pd.DataFrame(columns=columns)
row = {}
templen=len(temp.find_all("tr"))
page_df_dict = {}
for i in range(2, templen):
    tempTr = temp.find_all("tr")[i]
    if(tempTr.find("th") is not None):
        continue

    column_idx = 0
    for j in range(len(columns)):
        tempTd = tempTr.find_all("td")[j].text.strip()
        if(tempTd == "" and j > 0):
            continue
        row[columns[column_idx]] = row.get(columns[column_idx],[''])+[tempTd]
        column_idx += 1
    page_df = pd.DataFrame(row).iloc[1:,:]
    
    #print(row)
    #df = pd.concat([df, pd.DataFrame(row)])


In [259]:
len(row['wOBA'])

31

In [126]:
page_df

,순,이름,팀,게임,타석,타수,득점,안타,2타,3타,...,고4,삼진,병살,희타,희비,타율,출루,장타,OPS,wOBA
1,1,노시환,23한3B,6.93,131,595,514,85,153,30,...,4,5,118,13,0,3,.298,.388,.541,.929
2,2,김혜성,23키2B,6.19,137,621,556,104,186,29,...,3,3,77,6,0,5,.335,.396,.446,.842
3,3,최정,23S3B,5.78,128,552,471,94,140,31,...,15,6,87,12,1,6,.297,.388,.548,.936
4,4,양의지,23두C,5.40,129,509,438,56,134,23,...,11,8,56,20,0,3,.306,.397,.475,.872
5,5,오스틴,23L1B,5.22,139,583,520,87,163,29,...,3,2,75,11,0,7,.313,.376,.517,.893
6,6,홍창기,23LRF,5.17,141,643,524,109,174,35,...,22,3,83,6,3,6,.332,.444,.412,.856
7,7,구자욱,23삼RF,4.78,119,515,453,65,152,37,...,4,2,81,5,2,3,.336,.407,.494,.902
8,8,소크라테스,23KCF,4.58,142,608,547,91,156,31,...,1,4,80,13,0,8,.285,.344,.463,.806
9,9,손아섭,23NDH,4.39,140,609,551,97,187,36,...,2,2,67,10,1,5,.339,.393,.443,.836
10,10,박건우,23NRF,4.25,130,533,458,70,146,34,...,9,0,71,14,1,9,.319,.397,.480,.877
